In [1]:
import requests
from bs4 import BeautifulSoup

## 네이버 카페글 검색/수집(거래글)

In [3]:
page_url = 'https://search.naver.com/search.naver'
params = 'where=articlec&ie=utf8&query=%EC%97%90%EC%96%B4%ED%8C%9F&prdtype=0&t=0&st=rel&date_option=0&date_from=&date_to=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&board=&sm=tab_btn'

In [5]:
params.split('&') # 쿼리스트링은 &로 쪼개 집니당

['where=articlec',
 'ie=utf8',
 'query=%EC%97%90%EC%96%B4%ED%8C%9F',
 'prdtype=0',
 't=0',
 'st=rel',
 'date_option=0',
 'date_from=',
 'date_to=',
 'srchby=text',
 'dup_remove=1',
 'cafe_url=',
 'without_cafe_url=',
 'board=',
 'sm=tab_btn']

In [8]:

params = {
    'where' : 'articlec', # 거래글, 일반글 이 모두 다르다.
    'query' : '에어팟',
}
res = requests.get(page_url, params = params)
res

<Response [200]>

In [9]:
html = res.text
soup = BeautifulSoup(html, 'html.parser') # 페이지 소스보기로 확인하고 진행하는거다

In [12]:
tag_list = soup.select('.sh_cafe_top')
len(tag_list)

10

In [ ]:
'https://search.naver.com/search.naver?where=articlec&ie=utf8&query=%EC%97%90%EC%96%B4%ED%8C%9F&prdtype=0&t=0&st=rel&date_option=0&date_from=&date_to=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&board=&sm=tab_pge&start=11'
'https://search.naver.com/search.naver?where=articlec&ie=utf8&query=%EC%97%90%EC%96%B4%ED%8C%9F&prdtype=0&t=0&st=rel&date_option=0&date_from=&date_to=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&board=&sm=tab_pge&start=21'

In [ ]:
# 스타트라는 인자만 바뀐것을 확인함.

In [97]:
검색어 ='에어팟'
page_url = 'https://search.naver.com/search.naver'
page = 0
params = {
    'where' : 'articlec', # 거래글, 일반글 이 모두 다르다.
    'query' : 검색어,
    'start' : (page-1) * 10 + 1,
}
res = requests.get(page_url, params = params)
html = res.text
soup = BeautifulSoup(html, 'html.parser')
tag_list = soup.select('.sh_cafe_top')
len(tag_list)

10

In [50]:
tag = tag_list[3]

In [51]:
title_tag = tag.select_one('.sh_cafe_title')
post_url = title_tag['href']
post_title = title_tag.text
post_url, post_title

('https://cafe.naver.com/appleiphone/5061001', '아이폰x64g + 에어팟 팝니다')

In [52]:
summary = tag.select_one('.sh_cafe_passage').text
summary

'아이폰x64g + 에어팟 팝니다, 휴대폰은 색상 특성상 옆면이 크롬재질이라 케이스 끼고벗길때 있는 흠집?말고는 없습니다 케이스 끼시면 안보입니다. 나머지는 깔끔 ...'

In [53]:
tag.select_one('.cafe_item_price .num').text

'750,000'

In [54]:
# is_sell = tag.select_one('.cafe_item_price del') is None
# is_sell 아래 방식이 더 좋아요~

True

In [56]:
is_sell = 'is_sell' in tag.select_one('.cafe_item_status')['class']
is_sell

False

In [62]:
cafe_tag = tag.select_one('.txt_block a[href*=cafe.naver.com]') # 카페명 뽑아오기
cafe_url = cafe_tag['href']
cafe_name = cafe_tag.text
cafe_url, cafe_name

('https://cafe.naver.com/appleiphone', '아사모 -[아이폰, 아이폰 Xr, Xs...')

In [66]:
# 날짜 가져오기
when = tag.select_one('.txt_inline').text
when

'어제 '

In [71]:
is_safe = tag.select_one('.txt_safe') is not None # 안전거래

In [75]:
img_url = tag.select_one('.thumb img')['src']

In [107]:
post_list = []
for tag in tag_list:
    title_tag = tag.select_one('.sh_cafe_title')
    post_url = title_tag['href']
    post_title = title_tag.text
    summary = tag.select_one('.sh_cafe_passage').text
    price = tag.select_one('.cafe_item_price .num').text
    is_sell = 'sell' in tag.select_one('.cafe_item_status')['class']
    cafe_tag = tag.select_one('.txt_block a[href*=cafe.naver.com]') # 카페명 뽑아오기
    cafe_url = cafe_tag['href']
    cafe_name = cafe_tag.text
    when = tag.select_one('.txt_inline').text
    is_safe = tag.select_one('.txt_safe') is not None
    img_url = tag.select_one('.thumb img')['src']
    
    post_list.append({
        'when' : when,
        'post_url' : post_url,
        'mobile_post_url' : post_url.replace('cafe.naver.com', 'm.cafe.naver.com'),
        'post_title' : post_title,
        'summary' : summary,
        'price' : price,
        'is_sell' : is_sell,
        'is_safe' : is_safe,
        'img_url' : img_url,
        'cafe_url' : cafe_url,
        'cafe_name' : cafe_name,
    })

In [108]:
#완료된거는 빼겠다!
for post in filter(lambda post: post['is_sell'], post_list):
    print('{when} {post_title} {price}원 {is_safe} {cafe_name}'.format(**post))
    print(post['mobile_post_url'])

어제  아이폰x64g + 에어팟 팝니다 750,000원 False 아사모 -[아이폰, 아이폰 Xr, Xs...
https://m.cafe.naver.com/appleiphone/5061001
3일 전  에어팟 투명하드케이스 3,000원 False 아사모 -[아이폰, 아이폰 Xr, Xs...
https://m.cafe.naver.com/appleiphone/5059254
2일 전  미개봉 에어팟 무선 충전케이스 팝니다 93,000원 False 아사모 -[아이폰, 아이폰 Xr, Xs...
https://m.cafe.naver.com/appleiphone/5059529


In [92]:
# 프레임이라는 태그로 중첨되어져 있어서 안보인다.
# 리퀘스트는 가자 바깥 부분엔 요청해서 처리한다. 페이스 소스보기에서 ifram
# m.을 붙여서 소스보기로 프레임을 볼 수 있다. 우리가 하는 식으로 볼 수 있다.

In [94]:
mobile_page_url = 'https://m.cafe.naver.com/appleiphone/5059232'

In [96]:
res = requests.get(mobile_page_url)
'카페 멤버만' in res.text

True

검색페이지를 거쳐서 들어가면 볼 수 있다

In [100]:
from urllib.parse import quote
quote('검색어')

'%EA%B2%80%EC%83%89%EC%96%B4'

In [101]:
mobile_page_url = 'https://m.cafe.naver.com/appleiphone/5059232'
headers = {
    'Referer' : 'https://m.cafe.naver.com/SectionMarketArticleSearch.nhn?page=1&sortBy=0&query='+quote('검색어')
}

In [104]:
res = requests.get(mobile_page_url, headers = headers)
'카페 멤버만' in res.text

False

성공!!